In [33]:
import matplotlib.pyplot as plt  
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.ticker as ticker
import fuzzywuzzy 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import sklearn

In [34]:
AQi_17 = pd.read_csv('AQI_17N.csv')
AQi_18 = pd.read_csv('AQI_18N.csv')
AQi_19 = pd.read_csv('AQI_19N.csv')
AQi_20 = pd.read_csv('AQI_20N.csv')

In [35]:
########################

In [36]:
UnemploymentT = pd.read_csv('UMEPLOYMENT1720.csv') 

In [37]:
########################

In [38]:
#AQi, data cleaning, and transformation. Merging Data sets. 
#First order of business, change the index to be set with city, drop CBSA code, drop year. 

In [59]:
AQi_Comb = pd.concat([AQi_17, AQi_18, AQi_19, AQi_20], ignore_index=True)

In [60]:
AQi_Comb

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2017,270,241,28,1,0,0,0,108,51,36,0,0,206,64,0
1,Alabama,Clay,2017,118,104,14,0,0,0,0,66,52,30,0,0,0,118,0
2,Alabama,Colbert,2017,283,265,18,0,0,0,0,63,48,37,0,0,218,65,0
3,Alabama,DeKalb,2017,359,329,30,0,0,0,0,80,50,39,0,0,315,44,0
4,Alabama,Elmore,2017,226,221,5,0,0,0,0,58,45,35,0,0,226,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4068,Wyoming,Sublette,2020,366,297,64,4,1,0,0,151,61,44,0,0,355,11,0
4069,Wyoming,Sweetwater,2020,366,238,116,7,4,0,1,1250,76,46,0,5,253,7,101
4070,Wyoming,Teton,2020,366,318,36,7,5,0,0,161,54,42,0,0,329,37,0
4071,Wyoming,Uinta,2020,366,323,41,2,0,0,0,122,51,40,0,2,225,0,139


In [61]:
AQi_N = AQi_Comb
AQi_N = AQi_N.drop('State', axis=1)
AQi_N = AQi_N.drop('Unhealthy for Sensitive Groups Days', axis=1)
AQi_N = AQi_N.drop('Unhealthy Days', axis=1)
AQi_N = AQi_N.drop('Very Unhealthy Days', axis=1)
AQi_N = AQi_N.drop('Hazardous Days', axis=1)
AQi_N = AQi_N.drop('Moderate Days', axis=1)
AQi_N = AQi_N.drop('Median AQI', axis=1)

In [62]:
AQi_N

,County,Year,Days with AQI,Good Days,Max AQI,90th Percentile AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Baldwin,2017,270,241,108,51,0,0,206,64,0
1,Clay,2017,118,104,66,52,0,0,0,118,0
2,Colbert,2017,283,265,63,48,0,0,218,65,0
3,DeKalb,2017,359,329,80,50,0,0,315,44,0
4,Elmore,2017,226,221,58,45,0,0,226,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4068,Sublette,2020,366,297,151,61,0,0,355,11,0
4069,Sweetwater,2020,366,238,1250,76,0,5,253,7,101
4070,Teton,2020,366,318,161,54,0,0,329,37,0
4071,Uinta,2020,366,323,122,51,0,2,225,0,139


In [ ]:
########################

In [63]:
UnemploymentT

,"Unemployment and median household income for the U.S., States, and counties, 2000-2021",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95
0,Sources: Unemployment: U.S. Department of Labo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"For definitions of rural classifications, see ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"This table was prepared by USDA, Economic Rese...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FIPS_code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Median_Household_Income_2020,Med_HH_Income_Percent_of_State_Total_2020
4,00000,US,United States,NaN,NaN,NaN,"142,601,576","136,904,853","5,696,723",4.0,...,"161,207,103","148,183,367","13,023,736",8.1,"161,758,337","153,099,687","8,658,650",5.4,"67,340",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276,72145,PR,"Vega Baja Municipio, PR",1,1,1,"19,805","17,607","2,198",11.1,...,NaN,NaN,NaN,NaN,"14,185","12,742","1,443",10.2,NaN,NaN
3277,72147,PR,"Vieques Municipio, PR",7,12,0,"2,441","2,120",321,13.2,...,NaN,NaN,NaN,NaN,"2,775","2,435",340,12.3,NaN,NaN
3278,72149,PR,"Villalba Municipio, PR",2,2,1,"7,636","6,648",988,12.9,...,NaN,NaN,NaN,NaN,"7,700","6,807",893,11.6,NaN,NaN
3279,72151,PR,"Yabucoa Municipio, PR",1,1,1,"10,613","9,005","1,608",15.2,...,NaN,NaN,NaN,NaN,"9,164","8,217",947,10.3,NaN,NaN


In [93]:
UnemploymentT_N = UnemploymentT
UnemploymentT_N = UnemploymentT_N.drop([UnemploymentT_N.index[0], UnemploymentT_N.index[1], UnemploymentT_N.index[2]])

In [101]:
UnemploymentT_N.columns = UnemploymentT_N.iloc[0]
UnemploymentT_N = UnemploymentT_N.drop([UnemploymentT_N.index[0]])

In [102]:
UnemploymentT_N

3,FIPS_code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Median_Household_Income_2020,Med_HH_Income_Percent_of_State_Total_2020
4,00000,US,United States,NaN,NaN,NaN,"142,601,576","136,904,853","5,696,723",4.0,...,"161,207,103","148,183,367","13,023,736",8.1,"161,758,337","153,099,687","8,658,650",5.4,"67,340",NaN
5,01000,AL,Alabama,NaN,NaN,NaN,"2,147,173","2,047,731","99,442",4.6,...,"2,268,309","2,121,149","147,160",6.5,"2,246,993","2,169,721","77,272",3.4,"53,958",100.0
7,01003,AL,"Baldwin County, AL",3,2,1,"69,979","67,370","2,609",3.7,...,"98,910","92,751","6,159",6.2,"99,427","96,481","2,946",3.0,"71,135",131.8
8,01005,AL,"Barbour County, AL",6,6,0,"11,449","10,812",637,5.6,...,"8,680","8,004",676,7.8,"8,197","7,728",469,5.7,"38,866",72.0
9,01007,AL,"Bibb County, AL",1,1,1,"8,623","8,160",463,5.4,...,"8,712","8,073",639,7.3,"8,560","8,262",298,3.5,"50,907",94.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276,72145,PR,"Vega Baja Municipio, PR",1,1,1,"19,805","17,607","2,198",11.1,...,NaN,NaN,NaN,NaN,"14,185","12,742","1,443",10.2,NaN,NaN
3277,72147,PR,"Vieques Municipio, PR",7,12,0,"2,441","2,120",321,13.2,...,NaN,NaN,NaN,NaN,"2,775","2,435",340,12.3,NaN,NaN
3278,72149,PR,"Villalba Municipio, PR",2,2,1,"7,636","6,648",988,12.9,...,NaN,NaN,NaN,NaN,"7,700","6,807",893,11.6,NaN,NaN
3279,72151,PR,"Yabucoa Municipio, PR",1,1,1,"10,613","9,005","1,608",15.2,...,NaN,NaN,NaN,NaN,"9,164","8,217",947,10.3,NaN,NaN


In [108]:
New_Unemp = UnemploymentT_N.drop(UnemploymentT_N.filter(regex='2000').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2001').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2002').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2003').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2004').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2005').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2006').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2007').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2008').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2009').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2010').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2011').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2012').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2013').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2014').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2015').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2016').columns, axis=1)
New_Unemp = New_Unemp.drop(New_Unemp.filter(regex='2021').columns, axis=1)

In [109]:
New_Unemp

3,FIPS_code,State,Area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2020,Med_HH_Income_Percent_of_State_Total_2020
4,00000,US,United States,"160,744,592","153,744,181","7,000,411",4.4,"162,039,458","155,727,518","6,311,940",...,"163,815,890","157,805,900","6,009,990",3.7,"161,207,103","148,183,367","13,023,736",8.1,"67,340",NaN
5,01000,AL,Alabama,"2,203,458","2,103,873","99,585",4.5,"2,240,477","2,153,080","87,397",...,"2,271,892","2,199,868","72,024",3.2,"2,268,309","2,121,149","147,160",6.5,"53,958",100.0
7,01003,AL,"Baldwin County, AL","93,152","89,286","3,866",4.2,"96,252","92,769","3,483",...,"98,921","96,044","2,877",2.9,"98,910","92,751","6,159",6.2,"71,135",131.8
8,01005,AL,"Barbour County, AL","8,415","7,914",501,6.0,"8,505","8,071",434,...,"8,637","8,292",345,4.0,"8,680","8,004",676,7.8,"38,866",72.0
9,01007,AL,"Bibb County, AL","8,648","8,261",387,4.5,"8,698","8,354",344,...,"8,772","8,488",284,3.2,"8,712","8,073",639,7.3,"50,907",94.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276,72145,PR,"Vega Baja Municipio, PR","13,414","11,746","1,668",12.4,"13,159","11,787","1,372",...,"13,174","11,912","1,262",9.6,NaN,NaN,NaN,NaN,NaN,NaN
3277,72147,PR,"Vieques Municipio, PR","3,027","2,589",438,14.5,"2,726","2,276",450,...,"2,551","2,368",183,7.2,NaN,NaN,NaN,NaN,NaN,NaN
3278,72149,PR,"Villalba Municipio, PR","7,557","6,090","1,467",19.4,"7,335","6,174","1,161",...,"7,494","6,309","1,185",15.8,NaN,NaN,NaN,NaN,NaN,NaN
3279,72151,PR,"Yabucoa Municipio, PR","9,000","7,512","1,488",16.5,"8,791","7,534","1,257",...,"8,841","7,686","1,155",13.1,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
New_Unemp = New_Unemp.rename(columns={'Area_name':'County'})

In [111]:
New_Unemp['County'] = New_Unemp['County'].str.lower()
AQi_N['County'] = AQi_N['County'].str.lower()

In [128]:
########################

In [129]:
#Use Fuzzy Wuzzy to match the city's and merge the datasets--
#--Then drop the data points in PLACES to match the AQI_CBSA. 

In [113]:
AQi_N['County'] = AQi_N['County'].apply(
  lambda x: process.extractOne(x, New_Unemp['County'], scorer=fuzz.partial_ratio)[0])

In [142]:
New_DF = pd.merge(New_Unemp, AQi_N, on='County', how='left')

In [144]:
AQi_N.corr(New_Unemp)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().